# Chapter 5: Dependency parsing
https://nlp100.github.io/en/ch05.html The zip archive ai.en.zip contains the text of the Wikipedia article, “Artificial Intelligence”. Apply a dependency parser to the text, and store the result in a file. Implement programs that read the dependency trees and perform the jobs.

For your convenience, the zip archive also includes ai.en.txt.json, the text with dependency trees predicted by Stanford CoreNLP and stored in JSON format.

<hr>

## 40. Read the parse result (words)

Design a class Word that represents a word. This class has three member variables, text (word surface), lemma (lemma), and pos (part-of-speech). Represent a sentence as an array of instances of Word class. Implement a program to load the parse result, and store the text as an array of sentences. Show the object of the first sentence of the body of the article.

In [2]:
import re
import pandas as pd
import json

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

basePath = "/content/drive/MyDrive/nlp100/Dataset/alice/"
rawtext = "alice.txt"
conll = "alice.txt.conll"
jsn = 'alice.txt.json'

f = open(basePath + rawtext, 'r', encoding = "ISO-8859-1")
aliceRaw = f.read()
aliceRaw = re.sub(r"\n", r" ", aliceRaw)
aliceRaw = aliceRaw.strip()
aliceRawPerSentence = aliceRaw.split(".")

f = open(basePath + jsn, 'r')
aliceJson = json.load(f)

Mounted at /content/drive


In [3]:
aliceRaw[:500]

"ALICE'S ADVENTURES IN WONDERLAND  Lewis Carroll  THE MILLENNIUM FULCRUM EDITION 3.0     CHAPTER I. Down the Rabbit-Hole  Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversations?'  So she was considering in her own mind (as well as she could, for the hot d"

In [4]:
class Alice:
  def __init__(self, pathJsn):
    self._sentences = []
    self._aliceJson = []

    f = open(basePath + jsn, 'r')
    self._aliceJson = json.load(f)

    currentSentence = []

    for i, sentence in enumerate(self._aliceJson['sentences']):
      for j, word in enumerate(sentence['tokens']):
        a = self.Word(
            text = word['word'],
            lemma = word['lemma'],
            pos = word['pos'],
        )
        
        currentSentence.append(a)

      self._sentences.append(currentSentence)
      currentSentence = []

  class Word:
    def __init__(self, text, lemma, pos):
      self._text = text
      self._lemma = lemma
      self._pos = pos

In [5]:
alice = Alice(basePath + jsn)

for index, word in enumerate(alice._sentences[0]):
  print("==== index",index,"====")
  print("text\t:", word._text)
  print("lemma\t:",word._lemma)
  print("pos\t:", word._pos)

==== index 0 ====
text	: ALICE
lemma	: ALICE
pos	: NNP
==== index 1 ====
text	: 'S
lemma	: 's
pos	: POS
==== index 2 ====
text	: ADVENTURES
lemma	: ADVENTURES
pos	: NNP
==== index 3 ====
text	: IN
lemma	: in
pos	: IN
==== index 4 ====
text	: WONDERLAND
lemma	: WONDERLAND
pos	: NNP


<hr>

## 41. Read the parse result (dependency)

In addition to problem 40, add three member variables head (a reference to the object of its syntactic governor), dep (dependency type to its governor), and children (a list of references to the syntactic dependents in the parse tree) to the class Word. Show the pairs of governors (parents) and their dependents (children) of the first sentence of the body of the article. Use the class Word in the rest of the problems in this chapter.

In [85]:
class Alice:
  def __init__(self, pathJsn):
    self._sentences = []
    self._aliceJson = []

    f = open(basePath + jsn, 'r')
    self._aliceJson = json.load(f)

    currentSentence = []

    for i, sentence in enumerate(self._aliceJson['sentences']):
      for j, word in enumerate(sentence['tokens']):
        a = self.Word(
            text = word['word'],
            lemma = word['lemma'],
            pos = word['pos'],
            head = sentence['enhancedPlusPlusDependencies'][j]['governorGloss'],
            headId = sentence['enhancedPlusPlusDependencies'][j]['governor'],
            dep = sentence['enhancedPlusPlusDependencies'][j]['dep'],
            children = sentence['enhancedPlusPlusDependencies'][j]['dependentGloss'],
            childrenId = sentence['enhancedPlusPlusDependencies'][j]['dependent']
        )
        
        currentSentence.append(a)

      self._sentences.append(currentSentence)
      currentSentence = []

  class Word:
    def __init__(self, text, lemma, pos, head, headId, dep, children, childrenId):
      self._text = text
      self._lemma = lemma
      self._pos = pos

      self._head = head
      self._headId = headId
      self._dep = dep
      self._children = children
      self._childrenId = childrenId

In [86]:
alice = Alice(basePath + jsn)

for index, word in enumerate(alice._sentences[0]):
  print("==== index",index,"====")
  print("text\t:", word._text)
  print("lemma\t:",word._lemma)
  print("pos\t:", word._pos)
  print("head\t:", word._head)
  print("dep\t:", word._dep)
  print("children:", word._children)

==== index 0 ====
text	: ALICE
lemma	: ALICE
pos	: NNP
head	: ROOT
dep	: ROOT
children: ADVENTURES
==== index 1 ====
text	: 'S
lemma	: 's
pos	: POS
head	: ADVENTURES
dep	: nmod:poss
children: ALICE
==== index 2 ====
text	: ADVENTURES
lemma	: ADVENTURES
pos	: NNP
head	: ALICE
dep	: case
children: 'S
==== index 3 ====
text	: IN
lemma	: in
pos	: IN
head	: WONDERLAND
dep	: case
children: IN
==== index 4 ====
text	: WONDERLAND
lemma	: WONDERLAND
pos	: NNP
head	: ADVENTURES
dep	: nmod:in
children: WONDERLAND


<hr>

## 42. Show root words
For each sentence, extract the root word (whose head is ROOT).

In [87]:
for i, sentence in enumerate(alice._sentences[:4]):
  for j, word in enumerate(sentence):
    if word._head == 'ROOT':
      print(word._text,"is the root of sentence ",i)

ALICE is the root of sentence  0
Lewis is the root of sentence  1
THE is the root of sentence  2
CHAPTER is the root of sentence  3


<hr>

## 43. Show verb governors and noun dependents
Show all pairs of verb governors (parents) and their noun dependents (children) from all sentences in the text.

In [88]:
for i, sentence in enumerate(alice._sentences[:4]):
  print("======= sentence",i,"=======")
  for j, word in enumerate(sentence):
    print("parent\t: ",word._head," \t\tchild\t: ",word._children)

======= sentence 0 =======
parent	:  ROOT  		child	:  ADVENTURES
parent	:  ADVENTURES  		child	:  ALICE
parent	:  ALICE  		child	:  'S
parent	:  WONDERLAND  		child	:  IN
parent	:  ADVENTURES  		child	:  WONDERLAND
======= sentence 1 =======
parent	:  ROOT  		child	:  Carroll
parent	:  Carroll  		child	:  Lewis
======= sentence 2 =======
parent	:  ROOT  		child	:  EDITION
parent	:  EDITION  		child	:  THE
parent	:  EDITION  		child	:  MILLENNIUM
parent	:  EDITION  		child	:  FULCRUM
parent	:  EDITION  		child	:  3.0
======= sentence 3 =======
parent	:  ROOT  		child	:  I.
parent	:  I.  		child	:  CHAPTER
parent	:  Rabbit-Hole  		child	:  Down
parent	:  Rabbit-Hole  		child	:  the
parent	:  I.  		child	:  Rabbit-Hole


<hr>

## 44. Visualize dependency trees
Visualize a dependency tree of a sentence as a directed graph. Consider converting a dependency tree into DOT language and use Graphviz for drawing a directed graph. In addition, you can use pydot for drawing a dependency tree.

In [ ]:
pip install treelib

In [89]:
from treelib import Node, Tree

In [90]:
for i, sentence in enumerate(alice._sentences[:4]):
  print("======= sentence",i,"=======")

  orderList = ['ROOT']
  mapping = [''] # string '' will be removed later. Only for indexing
  sentenceCopy = sentence.copy()
  while len(orderList) != len(sentence) + 1: # DFS-ordering as treelib needs the data to be ordered.
    for j, word in enumerate(sentenceCopy):
      if word._head in orderList and word._children not in orderList:
        indexInsert = orderList.index(word._head) + 1
        orderList.insert(indexInsert, word._children)
        mapping.insert(indexInsert, {'parent': word._head, 'child': word._children})
        sentenceCopy.pop(j)

  tree = Tree()
  mapping.pop(0)

  for i, v in enumerate(mapping):
    if i == 0:
      tree.create_node(v['child'], v['child'])
    else:
      tree.create_node(v['child'], v['child'], v['parent'])

  tree.show()

======= sentence 0 =======
ADVENTURES
├── ALICE
│   └── 'S
└── WONDERLAND
    └── IN

======= sentence 1 =======
Carroll
└── Lewis

======= sentence 2 =======
EDITION
├── 3.0
├── FULCRUM
├── MILLENNIUM
└── THE

======= sentence 3 =======
I.
├── CHAPTER
└── Rabbit-Hole
    ├── Down
    └── the



<hr>

## 45. Triple with subject, verb, and direct object
We are interested in extracting facts from the text. In this chapter, we represent a fact as a tuple of (subject, predicate, object). Extract tuples from dependency trees where:
*   subject is a nominal subject of a verb in the past tense
*   predicate is the verb in the past tense
*   object is a direct object of the verb

Consider an example sentence, “Frank Rosenblatt invented the perceptron”. We want to extract a tuple, (Rosenblatt, invented, perceptron), from the sentence. In this problem, we only consider a subject and object as a single word.

This graph shows a dependency tree for the sentence (this may vary depending on the parser).

<i> check the site for the image </i>

In order to extract a tuple from a dependency tree, it may be a good idea to design an extraction rule on the dependency tree, for example,

<i> check the site for the image </i>

### Rule: no restriction



In [98]:
for i, sentence in enumerate(alice._sentences):
  vbds = []
  vbdsId = [] # this Id is needed because some words occured multiple times (distinct each same word in a sentence)

  for j, word in enumerate(sentence):
    if word._pos == "VBD": 
      vbds.append(word._text)
      vbdsId.append(j+1)

  for e, vbd in enumerate(vbds):
    subj = []
    obj = []
    
    for j, word in enumerate(sentence):
      if word._head == vbd and word._headId == vbdsId[e] and word._text != word._head and word._text not in [",", "`", ";",".",":"]:
        if word._dep == "nsubj": subj.append(word._text)
        if word._dep == "dobj": obj.append(word._text)

    if subj and obj:
      print(i, subj, "----", vbd, "----", obj)


4 ['conversations'] ---- had ---- ['what']
9 ['actually'] ---- TOOK ---- ['OF']
12 ['she'] ---- had ---- ['time']
18 ['I'] ---- fell ---- ['top']
48 ['she'] ---- put ---- ['way', 'it']
55 ['she'] ---- turned ---- ['corner']
59 ['she'] ---- tried ---- ['key']
68 ['nice'] ---- had ---- ['cherry-tart']
82 ['generally'] ---- gave ---- ['advice']
82 ['she'] ---- followed ---- ['it']
82 ['she'] ---- scolded ---- ['herself']
89 ['same'] ---- remained ---- ['be']
110 ['she'] ---- took ---- ['hurried']
116 ['she'] ---- dried ---- ['eyes']
140 ['hands'] ---- crossed ---- ['lap']
155 ['just'] ---- dropped ---- ['to']
156 ['That'] ---- WAS ---- ['escape']
166 ['I'] ---- cried ---- ['much']
197 ['always'] ---- HATED ---- ['cats']
216 ['they'] ---- had ---- ['consultation']
254 ['she'] ---- wanted ---- ['much']
267 ['no'] ---- pulled ---- ['-LRB-']
275 ['Dodo'] ---- presented ---- ['thimble']
276 ['simply'] ---- took ---- ['looking']
278 ['over'] ---- begged ---- ['to']
305 ['only'] ---- shook ---- 

### Rule: subject must be NNP or NN


In [100]:
for i, sentence in enumerate(alice._sentences):
  vbds = []
  vbdsId = []

  for j, word in enumerate(sentence):
    if word._pos == "VBD": 
      vbds.append(word._text)
      vbdsId.append(j+1)

  for e, vbd in enumerate(vbds):
    subj = []
    obj = []
    
    for j, word in enumerate(sentence):
      if word._head == vbd and word._headId == vbdsId[e] and word._text != word._head and word._text not in [",", "`", ";",".",":"]:
        if word._dep == "nsubj" and word._pos in ["NNP","NN"]: subj.append(word._text)
        if word._dep == "dobj": obj.append(word._text)

    if subj and obj:
      print(i, subj, "----", vbd, "----", obj)


275 ['Dodo'] ---- presented ---- ['thimble']
307 ['Crab'] ---- took ---- ['opportunity']
366 ['effect'] ---- tried ---- ['with']
422 ['Bill'] ---- got ---- ['other']
472 ['stick'] ---- made ---- ['head']
472 ['side', 'game'] ---- ran ---- ['charges']
685 ['everybody'] ---- minded ---- ['business']
775 ['pig'] ---- replied ---- ['Alice']
927 ['Hatter'] ---- went ---- ['!']
941 ['Dormouse'] ---- followed ---- ['him']
941 ['Alice'] ---- took ---- ['of']
990 ['nothing'] ---- said ---- ['at']
1045 ['Alice'] ---- joined ---- ['procession']
1081 ['someone'] ---- had ---- ['listen']
1112 ['anything'] ---- had ---- ['could']
1400 ['breath'] ---- drew ---- ['said']
1428 ['Panther'] ---- took ---- ['pie-crust']
1428 ['Owl'] ---- had ---- ['dish']
1430 ['Turtle'] ---- interrupted ---- ['What']
1436 ['Gryphon'] ---- said ---- ['that']
1488 ['mark'] ---- left ---- ['slate']
1491 ['Rabbit'] ---- blew ---- ['the']
1497 ['Rabbit'] ---- blew ---- ['on']
1507 ['jury'] ---- wrote ---- ['their']
1674 ['THE

### Rule: subject must be NNP



In [102]:
for i, sentence in enumerate(alice._sentences):
  vbds = []
  vbdsId = []

  for j, word in enumerate(sentence):
    if word._pos == "VBD": 
      vbds.append(word._text)
      vbdsId.append(j+1)

  for e, vbd in enumerate(vbds):
    subj = []
    obj = []
    
    for j, word in enumerate(sentence):
      if word._head == vbd and word._headId == vbdsId[e] and word._text != word._head and word._text not in [",", "`", ";",".",":"]:
        if word._dep == "nsubj" and word._pos == "NNP": subj.append(word._text)
        if word._dep == "dobj": obj.append(word._text)
  
    if subj and obj:
      print(i, subj, "----", vbd, "----", obj)


275 ['Dodo'] ---- presented ---- ['thimble']
307 ['Crab'] ---- took ---- ['opportunity']
422 ['Bill'] ---- got ---- ['other']
941 ['Alice'] ---- took ---- ['of']
1045 ['Alice'] ---- joined ---- ['procession']
1430 ['Turtle'] ---- interrupted ---- ['What']
1436 ['Gryphon'] ---- said ---- ['that']
1491 ['Rabbit'] ---- blew ---- ['the']
1497 ['Rabbit'] ---- blew ---- ['on']
1674 ['THEY'] ---- GAVE ---- ['HIM']
1675 ['HIM'] ---- said ---- ['Alice']


<hr>

## 46. Expanding subjects and objects
Improve the program of Problem 45 to remove the restriction that subjects and objects are single words but can also be phrases. For example, we want to extract (Frank Rosenblatt, invented, perceptron) from the sentence, “Frank Rosenblatt invented the perceptron”.

### Rule: no restriction

In [104]:
for i, sentence in enumerate(alice._sentences):
  vbds = []
  vbdsId = []

  for j, word in enumerate(sentence):
    if word._pos == "VBD": 
      vbds.append(word._text)
      vbdsId.append(j+1)

  for e, vbd in enumerate(vbds):
    subj = []
    obj = []
    
    for j, word in enumerate(sentence):
      if word._head == vbd and word._headId == vbdsId[e] and word._text != word._head and word._text not in [",", "`", ";",".",":"] :
        if word._dep in ["nsubj", "dobj"]:
          phrase = []
          phrase.append(word._text)
          prevIndex = j - 1
          nextIndex = j + 1
          
          #lookback sub
          currentWord = word._text
          while prevIndex > 0 and sentence[prevIndex]._head == currentWord and sentence[prevIndex]._dep == "compound":
            phrase.insert(0, sentence[prevIndex]._text)
            currentWord = sentence[prevIndex]._text
            prevIndex -= prevIndex

          #lookfront sub
          currentWord = word._text
          if nextIndex < len(sentence):
            while sentence[nextIndex]._head == currentWord and sentence[nextIndex]._dep == "compound":
              phrase.append(sentence[nextIndex]._text)
              currentWord = sentence[nextIndex]._text
              nextIndex += nextIndex

        if word._dep == "nsubj": 
          subj.append(" ".join(phrase))
        elif word._dep == "dobj":
          obj.append(" ".join(phrase))

    if subj and obj:
      print(i, subj, "----", vbd, "----", obj)



4 ['conversations'] ---- had ---- ['what']
9 ['actually'] ---- TOOK ---- ['OF']
12 ['she'] ---- had ---- ['time']
18 ['I'] ---- fell ---- ['top']
48 ['she'] ---- put ---- ['way', 'it']
55 ['she'] ---- turned ---- ['corner']
59 ['she'] ---- tried ---- ['key']
68 ['nice'] ---- had ---- ['cherry-tart']
82 ['generally'] ---- gave ---- ['advice']
82 ['she'] ---- followed ---- ['it']
82 ['she'] ---- scolded ---- ['herself']
89 ['same'] ---- remained ---- ['be']
110 ['she'] ---- took ---- ['hurried']
116 ['she'] ---- dried ---- ['eyes']
140 ['hands'] ---- crossed ---- ['lap']
155 ['just'] ---- dropped ---- ['to']
156 ['That'] ---- WAS ---- ['escape']
166 ['I'] ---- cried ---- ['much']
197 ['always'] ---- HATED ---- ['cats']
216 ['they'] ---- had ---- ['consultation']
254 ['she'] ---- wanted ---- ['much']
267 ['no'] ---- pulled ---- ['-LRB-']
275 ['Dodo'] ---- presented ---- ['thimble']
276 ['simply'] ---- took ---- ['looking']
278 ['over'] ---- begged ---- ['to']
305 ['only'] ---- shook ---- 

### Rule: subject must be NN or NNP

In [106]:
for i, sentence in enumerate(alice._sentences):
  vbds = []
  vbdsId = []

  for j, word in enumerate(sentence):
    if word._pos == "VBD": 
      vbds.append(word._text)
      vbdsId.append(j+1)

  for e, vbd in enumerate(vbds):
    subj = []
    obj = []
    
    for j, word in enumerate(sentence):
      if word._head == vbd and word._headId == vbdsId[e] and word._text != word._head and word._text not in [",", "`", ";",".",":"]:
        if word._dep == "nsubj" and word._pos in ["NNP", "NN"]: 
          phrase = []
          phrase.append(word._text)
          prevIndex = j - 1
          nextIndex = j + 1
          
          #lookback sub
          currentWord = word._text
          while prevIndex > 0 and sentence[prevIndex]._head == currentWord and sentence[prevIndex]._dep == "compound":
            phrase.insert(0, sentence[prevIndex]._text)
            currentWord = sentence[prevIndex]._text
            prevIndex -= prevIndex

          #lookfront sub
          currentWord = word._text
          if nextIndex < len(sentence):
            while sentence[nextIndex]._head == currentWord and sentence[nextIndex]._dep == "compound":
              phrase.append(sentence[nextIndex]._text)
              currentWord = sentence[nextIndex]._text
              nextIndex += nextIndex

          subj.append(" ".join(phrase))

        if word._dep == "dobj":
          phrase = []
          phrase.append(word._text)
          prevIndex = j - 1
          nextIndex = j + 1
          
          #lookback sub
          currentWord = word._text
          while prevIndex > 0 and sentence[prevIndex]._head == currentWord and sentence[prevIndex]._dep == "compound":
            phrase.insert(0, sentence[prevIndex]._text)
            currentWord = sentence[prevIndex]._text
            prevIndex -= prevIndex

          #lookfront sub
          currentWord = word._text
          if nextIndex < len(sentence):
            while sentence[nextIndex]._head == currentWord and sentence[nextIndex]._dep == "compound":
              phrase.append(sentence[nextIndex]._text)
              currentWord = sentence[nextIndex]._text
              nextIndex += nextIndex

          obj.append(" ".join(phrase))

    if subj and obj:
      print(i, subj, "----", vbd, "----", obj)


275 ['Dodo'] ---- presented ---- ['thimble']
307 ['Crab'] ---- took ---- ['opportunity']
366 ['effect'] ---- tried ---- ['with']
422 ['Bill'] ---- got ---- ['other']
472 ['stick'] ---- made ---- ['head']
472 ['side', 'game'] ---- ran ---- ['charges']
685 ['everybody'] ---- minded ---- ['business']
775 ['pig'] ---- replied ---- ['Alice']
927 ['Hatter'] ---- went ---- ['!']
941 ['Dormouse'] ---- followed ---- ['him']
941 ['Alice'] ---- took ---- ['of']
990 ['nothing'] ---- said ---- ['at']
1045 ['Alice'] ---- joined ---- ['procession']
1081 ['someone'] ---- had ---- ['listen']
1112 ['anything'] ---- had ---- ['could']
1400 ['breath'] ---- drew ---- ['said']
1428 ['Panther'] ---- took ---- ['pie-crust']
1428 ['Owl'] ---- had ---- ['dish']
1430 ['Mock Turtle'] ---- interrupted ---- ['What']
1436 ['Gryphon'] ---- said ---- ['that']
1488 ['mark'] ---- left ---- ['slate']
1491 ['White Rabbit'] ---- blew ---- ['the']
1497 ['White Rabbit'] ---- blew ---- ['on']
1507 ['jury'] ---- wrote ---- ['t

### Rule: subject must be NNP


In [105]:
for i, sentence in enumerate(alice._sentences):
  vbds = []
  vbdsId = []

  for j, word in enumerate(sentence):
    if word._pos == "VBD": 
      vbds.append(word._text)
      vbdsId.append(j+1)

  for e, vbd in enumerate(vbds):
    subj = []
    obj = []
    
    for j, word in enumerate(sentence):
      if word._head == vbd and word._headId == vbdsId[e] and word._text != word._head and word._text not in [",", "`", ";",".",":"]:
        if word._dep == "nsubj" and word._pos == "NNP": 
          phrase = []
          phrase.append(word._text)
          prevIndex = j - 1
          nextIndex = j + 1
          
          #lookback sub
          currentWord = word._text
          while prevIndex > 0 and sentence[prevIndex]._head == currentWord and sentence[prevIndex]._dep == "compound":
            phrase.insert(0, sentence[prevIndex]._text)
            currentWord = sentence[prevIndex]._text
            prevIndex -= prevIndex

          #lookfront sub
          currentWord = word._text
          if nextIndex < len(sentence):
            while sentence[nextIndex]._head == currentWord and sentence[nextIndex]._dep == "compound":
              phrase.append(sentence[nextIndex]._text)
              currentWord = sentence[nextIndex]._text
              nextIndex += nextIndex

          subj.append(" ".join(phrase))

        if word._dep == "dobj":
          phrase = []
          phrase.append(word._text)
          prevIndex = j - 1
          nextIndex = j + 1
          
          #lookback sub
          currentWord = word._text
          while prevIndex > 0 and sentence[prevIndex]._head == currentWord and sentence[prevIndex]._dep == "compound":
            phrase.insert(0, sentence[prevIndex]._text)
            currentWord = sentence[prevIndex]._text
            prevIndex -= prevIndex

          #lookfront sub
          currentWord = word._text
          if nextIndex < len(sentence):
            while sentence[nextIndex]._head == currentWord and sentence[nextIndex]._dep == "compound":
              phrase.append(sentence[nextIndex]._text)
              currentWord = sentence[nextIndex]._text
              nextIndex += nextIndex

          obj.append(" ".join(phrase))

    if subj and obj:
      print(i, subj, "----", vbd, "----", obj)


275 ['Dodo'] ---- presented ---- ['thimble']
307 ['Crab'] ---- took ---- ['opportunity']
422 ['Bill'] ---- got ---- ['other']
941 ['Alice'] ---- took ---- ['of']
1045 ['Alice'] ---- joined ---- ['procession']
1430 ['Mock Turtle'] ---- interrupted ---- ['What']
1436 ['Gryphon'] ---- said ---- ['that']
1491 ['White Rabbit'] ---- blew ---- ['the']
1497 ['White Rabbit'] ---- blew ---- ['on']
1674 ['THEY'] ---- GAVE ---- ['HIM']
1675 ['HIM'] ---- said ---- ['Alice']


<hr>

## 47. Triple from the passive sentence
Extract facts from sentences in the passive voice. Consider an example sentence, “Artificial intelligence was founded as an academic discipline in 1955”. We want to extract two tuples from the sentence,

* (Artificial intelligence, founded-as, academic discipline)
* (Artificial intelligence, founded-in, 1955)

<hr>

## 48. Extract paths from the root to nouns
For every noun in a dependency tree, extract a path from the root to the noun. Here, each path must satisfy the following specifications.

* Nodes in a path are words in surface form
* Nodes are connected with “ -> “ from the root to the leaf node
* We don’t have to include dependency types (e.g., nsubj, dobj) when representing a dependency path.

For the example sentence, “Frank Rosenblatt invented the perceptron”, we expect an output,

<i> check the site for the image </i>

<hr>

## 49. Extract the shortest path between two nouns
Extract the shortest path for every pair of two nouns. Supposing that two nouns appear at the i-th and j-th positions (in words) in a sentence (i<j), the shortest path must satisfy the following specifications.

* Nodes in a path are words in surface form
* Nodes corresponding to the i-th and j-th words are replaced with X and Y, respectively.
* Nodes are connected with either “ -> “ or “ <- “ from X to Y to represent a direction of a dependency.

We can consider two types of dependency paths.

* When the j-th word appears on the path from the i-th word to the root: the path from the i-th word to the j-th word
* When the i-th and j-th words have the common ancestor (the k-th word) in the dependency tree: the path from the i-th word to the k-th word connected with “ <- “, followed by the path from the k-th word to the j-th word connected with “ -> “.

For the example sentence, “Frank Rosenblatt invented the perceptron”, we expect an output,

<i> check the site for the image </i>